In [1]:
import pandas as pd
import numpy as np
from sklearn import  preprocessing
import xgboost as xgb
#import lightgbm as lgb    

/home/silence/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df=pd.read_csv('Data/ccf_first_round_user_shop_behavior.csv')
shop=pd.read_csv('Data/ccf_first_round_shop_info.csv')
test=pd.read_csv('Data/evaluation_public.csv')

In [3]:
df.head()

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos
0,u_376,s_2871718,2017-08-06 21:20,122.308291,32.088040,b_6396480|-67|false;b_41124514|-86|false;b_287...
1,u_376,s_2871718,2017-08-06 21:20,122.308162,32.087970,b_6396480|-67|false;b_56328155|-73|false;b_411...
2,u_1041,s_181637,2017-08-02 13:10,117.365255,40.638214,b_8006367|-78|false;b_2485110|-52|false;b_3005...
3,u_1158,s_609470,2017-08-13 12:30,121.134451,31.197416,b_26250579|-73|false;b_26250580|-64|false;b_26...
4,u_1654,s_3816766,2017-08-25 19:50,122.255867,31.351320,b_39004150|-66|false;b_39004148|-58|false;b_21...


In [4]:
df=pd.merge(df,shop[['shop_id','mall_id']],how='left',on='shop_id')

In [5]:
df['time_stamp']=pd.to_datetime(df['time_stamp'])

In [6]:
df.head()

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos,mall_id
0,u_376,s_2871718,2017-08-06 21:20:00,122.308291,32.088040,b_6396480|-67|false;b_41124514|-86|false;b_287...,m_1409
1,u_376,s_2871718,2017-08-06 21:20:00,122.308162,32.087970,b_6396480|-67|false;b_56328155|-73|false;b_411...,m_1409
2,u_1041,s_181637,2017-08-02 13:10:00,117.365255,40.638214,b_8006367|-78|false;b_2485110|-52|false;b_3005...,m_4079
3,u_1158,s_609470,2017-08-13 12:30:00,121.134451,31.197416,b_26250579|-73|false;b_26250580|-64|false;b_26...,m_6587
4,u_1654,s_3816766,2017-08-25 19:50:00,122.255867,31.351320,b_39004150|-66|false;b_39004148|-58|false;b_21...,m_3005


In [7]:
test.head()

,row_id,user_id,mall_id,time_stamp,longitude,latitude,wifi_infos
0,118742,u_30097142,m_3916,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...
1,118743,u_30097803,m_5085,2017-09-06 13:10,118.191907,32.855858,b_36722251|-81|false;b_10537579|-75|false;b_43...
2,118744,u_30097889,m_4033,2017-09-06 17:40,119.192110,32.424667,b_30026291|-74|false;b_30026290|-74|false;b_36...
3,118745,u_30098996,m_4515,2017-09-03 12:10,120.612201,34.055249,b_33412374|-77|false;b_22084893|-86|false;b_52...
4,118746,u_30099170,m_7168,2017-09-02 20:40,116.861989,40.326858,b_19882704|-77|false;b_2241462|-49|false;b_585...


In [8]:
train=pd.concat([df,test])

In [9]:
train.head()

,latitude,longitude,mall_id,row_id,shop_id,time_stamp,user_id,wifi_infos
0,32.088040,122.308291,m_1409,NaN,s_2871718,2017-08-06 21:20:00,u_376,b_6396480|-67|false;b_41124514|-86|false;b_287...
1,32.087970,122.308162,m_1409,NaN,s_2871718,2017-08-06 21:20:00,u_376,b_6396480|-67|false;b_56328155|-73|false;b_411...
2,40.638214,117.365255,m_4079,NaN,s_181637,2017-08-02 13:10:00,u_1041,b_8006367|-78|false;b_2485110|-52|false;b_3005...
3,31.197416,121.134451,m_6587,NaN,s_609470,2017-08-13 12:30:00,u_1158,b_26250579|-73|false;b_26250580|-64|false;b_26...
4,31.351320,122.255867,m_3005,NaN,s_3816766,2017-08-25 19:50:00,u_1654,b_39004150|-66|false;b_39004148|-58|false;b_21...


In [11]:
train.tail()

,latitude,longitude,mall_id,row_id,shop_id,time_stamp,user_id,wifi_infos
483926,36.899323,117.256120,m_7523,483927.0,NaN,2017-09-10 13:50,u_63919729,b_30877374|-71|false;b_8210864|-61|false;b_308...
483927,40.495294,116.941969,m_4828,483928.0,NaN,2017-09-10 13:50,u_63920157,b_8685813|-79|false;b_8685969|-74|false;b_1621...
483928,40.497449,116.941186,m_4828,483929.0,NaN,2017-09-13 14:30,u_63920157,b_49160464|-65|false;b_34498163|-82|false;b_45...
483929,40.497446,116.941283,m_4828,483930.0,NaN,2017-09-11 12:10,u_63920157,b_45089046|-47|true;b_49160464|-65|false;b_397...
483930,32.753948,119.874121,m_7973,483931.0,NaN,2017-09-10 18:10,u_63920806,b_25163301|-69|false;b_39872252|-57|false;b_25...


In [12]:
mall_list=list(set(list(shop.mall_id))) #通过set和list互相转换达到去重的目的

In [13]:
mall_list # 97个商场

['m_4033',
 'm_5076',
 'm_6587',
 'm_5825',
 'm_690',
 'm_8379',
 'm_1409',
 'm_1831',
 'm_3916',
 'm_626',
 'm_9068',
 'm_1085',
 'm_6337',
 'm_2058',
 'm_3832',
 'm_4459',
 'm_4168',
 'm_7800',
 'm_3054',
 'm_3313',
 'm_2009',
 'm_2182',
 'm_1621',
 'm_4515',
 'm_4121',
 'm_7601',
 'm_3005',
 'm_3871',
 'm_4828',
 'm_3839',
 'm_5529',
 'm_1790',
 'm_3501',
 'm_8344',
 'm_3112',
 'm_3019',
 'm_5154',
 'm_5892',
 'm_4079',
 'm_1377',
 'm_1920',
 'm_4187',
 'm_2907',
 'm_4094',
 'm_3517',
 'm_5767',
 'm_8093',
 'm_822',
 'm_1021',
 'm_7973',
 'm_2333',
 'm_2415',
 'm_4548',
 'm_3425',
 'm_2878',
 'm_5352',
 'm_5810',
 'm_3739',
 'm_2224',
 'm_1293',
 'm_979',
 'm_4495',
 'm_4543',
 'm_3445',
 'm_4422',
 'm_622',
 'm_1263',
 'm_4406',
 'm_4011',
 'm_1089',
 'm_7374',
 'm_2715',
 'm_4341',
 'm_4923',
 'm_3528',
 'm_5085',
 'm_4572',
 'm_7168',
 'm_968',
 'm_1950',
 'm_625',
 'm_2123',
 'm_623',
 'm_1175',
 'm_6803',
 'm_2578',
 'm_7523',
 'm_909',
 'm_2267',
 'm_615',
 'm_6167',
 'm_4759'

In [14]:
result=pd.DataFrame()

In [15]:
mall = mall_list[0] #分mall进行训练

In [16]:
train_tmp = train[train.mall_id == mall].reset_index(drop=True)

In [17]:
train_tmp.head()

,latitude,longitude,mall_id,row_id,shop_id,time_stamp,user_id,wifi_infos
0,32.424672,119.192098,m_4033,NaN,s_21049,2017-08-20 11:50:00,u_123031,b_36629923|-65|false;b_30026290|-76|false;b_24...
1,32.424576,119.194926,m_4033,NaN,s_408779,2017-08-20 13:30:00,u_123031,b_26626440|-87|false;b_15021211|-81|false;b_20...
2,32.424728,119.192320,m_4033,NaN,s_296592,2017-08-27 14:50:00,u_217386,b_6484828|-58|false;b_39409154|-52|false;b_473...
3,32.424204,119.191428,m_4033,NaN,s_815600,2017-08-04 20:40:00,u_289614,b_30365357|-74|false;b_50234552|-45|false;b_54...
4,32.424698,119.192318,m_4033,NaN,s_296592,2017-08-22 15:10:00,u_468695,b_15009075|-66|false;b_224598|-80|false;b_4123...


In [18]:
l = [] # 存储元素为wifi_id和强度的字典，即把每一行的wifi信息存储为一个字典
wifi_dict = {}  # 该商场所有的wifi出现次数
for index,row in train_tmp.iterrows():
    r = {}
    wifi_list = [wifi.split('|') for wifi in row['wifi_infos'].split(';')]
    for i in wifi_list:
        r[i[0]] = int(i[1]) # wifi id 和强度的字典
        if i[0] not in wifi_dict:
            wifi_dict[i[0]] = 1
        else:
            wifi_dict[i[0]] += 1
    l.append(r)

In [19]:
filter_wifi = [] # 将出现次数小于15次的wifi过滤掉
for i in wifi_dict:
    if wifi_dict[i] < 15:
        filter_wifi.append(i)

m = []
for row in l:
    new = {}
    for n in row.keys():
        if n not in filter_wifi:
            new[n] = row[n]
    m.append(new)

In [33]:
m.count()

TypeError: count() takes exactly one argument (0 given)

In [20]:
train_tmp = pd.concat([train_tmp, pd.DataFrame(m)], axis=1)

In [32]:
train_tmp

,b_10499391,b_10513554,b_10562491,b_10562492,b_10964851,b_11230525,b_11321183,b_11436753,b_11439747,b_11703110,...,b_9521208,b_9795004,latitude,longitude,mall_id,row_id,shop_id,time_stamp,user_id,wifi_infos
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,32.424672,119.192098,m_4033,NaN,s_21049,2017-08-20 11:50:00,u_123031,b_36629923|-65|false;b_30026290|-76|false;b_24...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,32.424576,119.194926,m_4033,NaN,s_408779,2017-08-20 13:30:00,u_123031,b_26626440|-87|false;b_15021211|-81|false;b_20...
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,32.424728,119.192320,m_4033,NaN,s_296592,2017-08-27 14:50:00,u_217386,b_6484828|-58|false;b_39409154|-52|false;b_473...
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,32.424204,119.191428,m_4033,NaN,s_815600,2017-08-04 20:40:00,u_289614,b_30365357|-74|false;b_50234552|-45|false;b_54...
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,32.424698,119.192318,m_4033,NaN,s_296592,2017-08-22 15:10:00,u_468695,b_15009075|-66|false;b_224598|-80|false;b_4123...
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,32.424641,119.192315,m_4033,NaN,s_296592,2017-08-20 20:10:00,u_608003,b_45408600|-67|false;b_45408599|-46|false;b_30...
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,32.424649,119.192308,m_4033,NaN,s_296592,2017-08-29 12:10:00,u_608048,b_45197486|-72|false;b_47387710|-46|false;b_47...
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,32.424710,119.192111,m_4033,NaN,s_202391,2017-08-30 10:40:00,u_688454,b_28015120|-61|false;b_17049290|-65|false;b_30...
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,32.424665,119.192285,m_4033,NaN,s_296592,2017-08-20 09:50:00,u_777497,b_45408599|-40|false;b_40924464|-64|false;b_45...
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,32.424694,119.192725,m_4033,NaN,s_12521,2017-08-06 17:10:00,u_777497,b_39335939|-53|false;b_18769848|-77|false;b_21...


In [26]:
df_train = train_tmp[train_tmp.shop_id.notnull()]
df_test = train_tmp[train_tmp.shop_id.isnull()]

In [27]:
lbl = preprocessing.LabelEncoder() # 标签编码
lbl.fit(list(df_train['shop_id'].values))
df_train['label'] = lbl.transform(list(df_train['shop_id'].values))    

num_class = df_train['label'].max()+1    # 多分类类别数 

/home/silence/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
params = {
        'objective': 'multi:softmax',
        'eta': 0.1,
        'max_depth': 10,
        'eval_metric': 'merror',
        'seed': 4,
        'missing': -999,
        'num_class':num_class,
        'silent' : 1
        }

In [29]:
# 提取特征 只将wifi作为特征
feature=[x for x in train_tmp.columns if x not in ['user_id','label','shop_id','time_stamp','mall_id','wifi_infos']]    

xgbtrain = xgb.DMatrix(df_train[feature], df_train['label'])
xgbtest = xgb.DMatrix(df_test[feature])

In [30]:
watchlist = [ (xgbtrain,'train'), (xgbtrain, 'test') ]
num_rounds = 70
model = xgb.train(params, xgbtrain, num_rounds, watchlist, early_stopping_rounds=10) # 开始训练

[0]	train-merror:0.175221	test-merror:0.175221
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 10 rounds.
[1]	train-merror:0.160229	test-merror:0.160229
[2]	train-merror:0.15596	test-merror:0.15596
[3]	train-merror:0.150859	test-merror:0.150859
[4]	train-merror:0.147631	test-merror:0.147631
[5]	train-merror:0.143883	test-merror:0.143883
[6]	train-merror:0.142946	test-merror:0.142946
[7]	train-merror:0.14076	test-merror:0.14076
[8]	train-merror:0.138365	test-merror:0.138365
[9]	train-merror:0.136804	test-merror:0.136804
[10]	train-merror:0.135867	test-merror:0.135867
[11]	train-merror:0.135867	test-merror:0.135867
[12]	train-merror:0.136075	test-merror:0.136075
[13]	train-merror:0.135034	test-merror:0.135034
[14]	train-merror:0.133784	test-merror:0.133784
[15]	train-merror:0.132535	test-merror:0.132535
[16]	train-merror:0.132015	test-merror:0.132015
[17]	train-merror:0.130036	test-merror:0.130036
[18

In [31]:
df_test['label'] = model.predict(xgbtest)
df_test['shop_id'] = df_test['label'].apply(lambda x:lbl.inverse_transform(int(x))) #标签解码
r = df_test[['row_id','shop_id']]
result = pd.concat([result,r])
result['row_id'] = result['row_id'].astype('int')
result.to_csv('sub2.csv',index=False)

/home/silence/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/silence/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


ValueError: Cannot convert non-finite values (NA or inf) to integer

In [9]:
for mall in mall_list:
    train1=train[train.mall_id==mall].reset_index(drop=True)       
    l=[]
    wifi_dict = {}
    for index,row in train1.iterrows():
        r = {}
        wifi_list = [wifi.split('|') for wifi in row['wifi_infos'].split(';')]
        for i in wifi_list:
            r[i[0]]=int(i[1])
            if i[0] not in wifi_dict:
                wifi_dict[i[0]]=1
            else:
                wifi_dict[i[0]]+=1
        l.append(r)    
    delate_wifi=[]
    for i in wifi_dict:
        if wifi_dict[i]<17:
            delate_wifi.append(i)
    m=[]
    for row in l:
        new={}
        for n in row.keys():
            if n not in delate_wifi:
                new[n]=row[n]
        m.append(new)
    train1 = pd.concat([train1,pd.DataFrame(m)], axis=1)
    df_train=train1[train1.shop_id.notnull()]
    df_test=train1[train1.shop_id.isnull()]
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(df_train['shop_id'].values))
    df_train['label'] = lbl.transform(list(df_train['shop_id'].values))    
    num_class=df_train['label'].max()+1    
    
    params = {
            'objective': 'multi:softmax',
            'eta': 0.1,
            'max_depth': 10,
            'eval_metric': 'merror',
            'seed': 4,
            'missing': -999,
            'num_class':num_class,
            'silent' : 1
            }
    feature=[x for x in train1.columns if x not in ['user_id','label','shop_id','time_stamp','mall_id','wifi_infos']]    
    xgbtrain = xgb.DMatrix(df_train[feature], df_train['label'])
    xgbtest = xgb.DMatrix(df_test[feature])
    watchlist = [ (xgbtrain,'train'), (xgbtrain, 'test') ]
    num_rounds=70
    model = xgb.train(params, xgbtrain, num_rounds, watchlist, early_stopping_rounds=12)
    df_test['label']=model.predict(xgbtest)
    df_test['shop_id']=df_test['label'].apply(lambda x:lbl.inverse_transform(int(x)))
    r=df_test[['row_id','shop_id']]
    result=pd.concat([result,r])
    result['row_id']=result['row_id'].astype('int')
    result.to_csv('sub2.csv',index=False)
    print(cnt)
    print('/97')
    cnt += 1

KeyboardInterrupt: 